In [27]:
import os

In [28]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS'

In [33]:
os.chdir("/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan")

In [34]:
%pwd

'/Users/pavithrasenthilkumar/Documents/My_Codes/PROJECTS/Lung_Cancer_Classification_CT_Scan'

In [46]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [47]:
from src.lungClassifier.constants import *
from src.lungClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [48]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return training_config

In [49]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [53]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path,
            compile=False
        )
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

    def train_valid_generator(self):

        self.train_generator = tf.keras.utils.image_dataset_from_directory(
        directory=self.config.training_data,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        label_mode="categorical"
    )

        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=42,
            image_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            label_mode="categorical"
        )

        # Normalize pixel values (rescale to 0-1)
        normalization_layer = tf.keras.layers.Rescaling(1./255)

        self.train_generator = self.train_generator.map(lambda x, y: (normalization_layer(x), y))
        self.valid_generator = self.valid_generator.map(lambda x, y: (normalization_layer(x), y))

        # Optional: speed up pipeline
        AUTOTUNE = tf.data.AUTOTUNE
        self.train_generator = self.train_generator.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
        self.valid_generator = self.valid_generator.cache().prefetch(buffer_size=AUTOTUNE)

    # @staticmethod
    # def save_model(path: Path, model: tf.keras.Model):
    #     model.save(path)
    
    def save_model(self, path: Path, model: tf.keras.Model):
        # Save full model as a single .h5 file
        full_model_path = path.with_suffix(".h5")
        model.save(full_model_path)


    def train(self):
        # self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        # self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.steps_per_epoch = tf.data.experimental.cardinality(self.train_generator).numpy()
        self.validation_steps = tf.data.experimental.cardinality(self.valid_generator).numpy()      

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,  # example: Path("artifacts/training/model")
            model=self.model
        )

       

In [54]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-07-10 19:42:01,878: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-10 19:42:01,880: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-10 19:42:01,881: INFO: common: created directory at: artifacts]
[2025-07-10 19:42:01,881: INFO: common: created directory at: artifacts/training]
Found 302 files belonging to 2 classes.
Using 242 files for training.
Found 302 files belonging to 2 classes.
Using 60 files for validation.


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


16/16 ━━━━━━━━━━━━━━━━━━━━ 203s 13s/step - accuracy: 0.5057 - loss: 1.2957 - val_accuracy: 0.8833 - val_loss: 0.4155
[2025-07-10 19:45:25,280: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
